In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('MissingVals').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/22 11:53:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/22 11:53:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/22 11:53:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
df=spark.read.csv('wine.csv',header=True,inferSchema=True) #inferSchema --> age type turns from string to integer

In [6]:
df.show(5)
df.printSchema()

+---------------+--------------------+--------+-----+---------------+-----+---------+------+----------+--------------+-------+-----+--------------------+
|Winzer/Erzeuger|                Name|     Typ|Farbe|       Rebsorte|  Bio|# Gekauft| Preis|  Herkunft|        Region|Alkohol|notes|           Shop Link|
+---------------+--------------------+--------+-----+---------------+-----+---------+------+----------+--------------+-------+-----+--------------------+
|        Fiorini|Becco rosso Lambr...|Perlwein|  rot|      Lambrusco|false|        1|8,90 €|   Italien|Emilia Romagna| 11,50%| null|https://tuttovino...|
|        Fiorini|Re Gusto Lambrusc...|Perlwein|  rot|      Lambrusco|false|        9|8,90 €|   Italien|Emilia Romagna|  9,00%| null|https://tuttovino...|
|    Gschweicher|Grüner Veltliner ...|    Wein| weiß|Grüner Vetliner|false|        3|8,90 €|Österreich|   Weinviertel|  8,90%| null|https://tuttovino...|
|Gruber Röschitz|  Gelber Muskateller|    Wein| weiß|    Muskateller| true| 

In [8]:
df.describe().show()

+-------+---------------+-------------+--------+-----+----------+------------------+-------+----------+-----------+-------+-------+--------------------+
|summary|Winzer/Erzeuger|         Name|     Typ|Farbe|  Rebsorte|         # Gekauft|  Preis|  Herkunft|     Region|Alkohol|  notes|           Shop Link|
+-------+---------------+-------------+--------+-----+----------+------------------+-------+----------+-----------+-------+-------+--------------------+
|  count|             15|           15|      11|   13|        10|                11|     11|        13|         13|     13|      1|                  13|
|   mean|           null|         null|    null| null|      null| 2.909090909090909|   null|      null|       null|   null|   null|                null|
| stddev|           null|         null|    null| null|      null|2.3432688900143512|   null|      null|       null|   null|   null|                null|
|    min|        Barollo|Bardolino rot|Perlwein| rosé|Chardonnay|                 

## Drop Missing Values
### Drop Columns

In [9]:
df.na.drop().show()

+---------------+----+---+-----+--------+---+---------+-----+--------+------+-------+-----+---------+
|Winzer/Erzeuger|Name|Typ|Farbe|Rebsorte|Bio|# Gekauft|Preis|Herkunft|Region|Alkohol|notes|Shop Link|
+---------------+----+---+-----+--------+---+---------+-----+--------+------+-------+-----+---------+
+---------------+----+---+-----+--------+---+---------+-----+--------+------+-------+-----+---------+



In [10]:
df.drop('notes').na.drop().show()

+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+--------------------+
|Winzer/Erzeuger|                Name|     Typ|Farbe|       Rebsorte|  Bio|# Gekauft|  Preis|  Herkunft|        Region|Alkohol|           Shop Link|
+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+--------------------+
|        Fiorini|Becco rosso Lambr...|Perlwein|  rot|      Lambrusco|false|        1| 8,90 €|   Italien|Emilia Romagna| 11,50%|https://tuttovino...|
|        Fiorini|Re Gusto Lambrusc...|Perlwein|  rot|      Lambrusco|false|        9| 8,90 €|   Italien|Emilia Romagna|  9,00%|https://tuttovino...|
|    Gschweicher|Grüner Veltliner ...|    Wein| weiß|Grüner Vetliner|false|        3| 8,90 €|Österreich|   Weinviertel|  8,90%|https://tuttovino...|
|Gruber Röschitz|  Gelber Muskateller|    Wein| weiß|    Muskateller| true|        5| 9,40 €|Österreich|  

### Drop Rows

In [23]:
df.na.drop(how='any', thresh=3).show() # keep if there at least 3 non-null values

+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+-------+--------------------+
|Winzer/Erzeuger|                Name|     Typ|Farbe|       Rebsorte|  Bio|# Gekauft|  Preis|  Herkunft|        Region|Alkohol|  notes|           Shop Link|
+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+-------+--------------------+
|        Fiorini|Becco rosso Lambr...|Perlwein|  rot|      Lambrusco|false|        1| 8,90 €|   Italien|Emilia Romagna| 11,50%|   null|https://tuttovino...|
|        Fiorini|Re Gusto Lambrusc...|Perlwein|  rot|      Lambrusco|false|        9| 8,90 €|   Italien|Emilia Romagna|  9,00%|   null|https://tuttovino...|
|    Gschweicher|Grüner Veltliner ...|    Wein| weiß|Grüner Vetliner|false|        3| 8,90 €|Österreich|   Weinviertel|  8,90%|   null|https://tuttovino...|
|Gruber Röschitz|  Gelber Muskateller|    Wein| weiß|    M

In [25]:
df.na.drop(how='any', subset=['Typ']).show() # drop all rows where no 'Typ' is present

+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+-----+--------------------+
|Winzer/Erzeuger|                Name|     Typ|Farbe|       Rebsorte|  Bio|# Gekauft|  Preis|  Herkunft|        Region|Alkohol|notes|           Shop Link|
+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+-----+--------------------+
|        Fiorini|Becco rosso Lambr...|Perlwein|  rot|      Lambrusco|false|        1| 8,90 €|   Italien|Emilia Romagna| 11,50%| null|https://tuttovino...|
|        Fiorini|Re Gusto Lambrusc...|Perlwein|  rot|      Lambrusco|false|        9| 8,90 €|   Italien|Emilia Romagna|  9,00%| null|https://tuttovino...|
|    Gschweicher|Grüner Veltliner ...|    Wein| weiß|Grüner Vetliner|false|        3| 8,90 €|Österreich|   Weinviertel|  8,90%| null|https://tuttovino...|
|Gruber Röschitz|  Gelber Muskateller|    Wein| weiß|    Muskateller| 

## Fill missing values

In [26]:
df.na.fill('missing').show() # replace null with string missing

+---------------+--------------------+--------+-------+---------------+-----+---------+-------+----------+--------------+-------+-------+--------------------+
|Winzer/Erzeuger|                Name|     Typ|  Farbe|       Rebsorte|  Bio|# Gekauft|  Preis|  Herkunft|        Region|Alkohol|  notes|           Shop Link|
+---------------+--------------------+--------+-------+---------------+-----+---------+-------+----------+--------------+-------+-------+--------------------+
|        Fiorini|Becco rosso Lambr...|Perlwein|    rot|      Lambrusco|false|        1| 8,90 €|   Italien|Emilia Romagna| 11,50%|missing|https://tuttovino...|
|        Fiorini|Re Gusto Lambrusc...|Perlwein|    rot|      Lambrusco|false|        9| 8,90 €|   Italien|Emilia Romagna|  9,00%|missing|https://tuttovino...|
|    Gschweicher|Grüner Veltliner ...|    Wein|   weiß|Grüner Vetliner|false|        3| 8,90 €|Österreich|   Weinviertel|  8,90%|missing|https://tuttovino...|
|Gruber Röschitz|  Gelber Muskateller|    Wein

In [31]:
from pyspark.ml.feature import Imputer

cols = ['# Gekauft']
imputer = Imputer(
    inputCols=cols, 
    outputCols=["{}_imputed".format(c) for c in cols]
    ).setStrategy("median")



In [32]:
imputer.fit(df_pyspark).transform(df_pyspark).show()

+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+-------+--------------------+-----------------+
|Winzer/Erzeuger|                Name|     Typ|Farbe|       Rebsorte|  Bio|# Gekauft|  Preis|  Herkunft|        Region|Alkohol|  notes|           Shop Link|# Gekauft_imputed|
+---------------+--------------------+--------+-----+---------------+-----+---------+-------+----------+--------------+-------+-------+--------------------+-----------------+
|        Fiorini|Becco rosso Lambr...|Perlwein|  rot|      Lambrusco|false|        1| 8,90 €|   Italien|Emilia Romagna| 11,50%|   null|https://tuttovino...|                1|
|        Fiorini|Re Gusto Lambrusc...|Perlwein|  rot|      Lambrusco|false|        9| 8,90 €|   Italien|Emilia Romagna|  9,00%|   null|https://tuttovino...|                9|
|    Gschweicher|Grüner Veltliner ...|    Wein| weiß|Grüner Vetliner|false|        3| 8,90 €|Österreich|   Weinviertel|  8,90